## Connect to KeyVault

In [1]:
import os
from azure.keyvault.secrets import SecretClient
from azure.identity import DefaultAzureCredential

keyVaultName = "gios-keyvault"
KVUri = f"https://gios-keyvault.vault.azure.net/"

credential = DefaultAzureCredential()
client = SecretClient(vault_url=KVUri, credential=credential)

print(f"Retrieving your secrets from {keyVaultName}.")

os.environ["AZURE_OPENAI_API_KEY"] = client.get_secret("AZURE-OPENAI-API-KEY").value
os.environ["AZURE_OPENAI_ENDPOINT"] = client.get_secret("AZURE-OPENAI-ENDPOINT").value

os.environ["AZURE_SEARCH_API_KEY"] = client.get_secret("AZURE-SEARCH-API-KEY").value
os.environ["AZURE_SEARCH_SERVICE_ENDPOINT"] = client.get_secret("AZURE-SEARCH-ENDPOINT").value

print("Done")

Retrieving your secrets from gios-keyvault.
Done


In [2]:
import openai
import re
import requests
import sys
from num2words import num2words
import os
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding, cosine_similarity
from transformers import GPT2TokenizerFast

API_KEY = os.getenv("AZURE_OPENAI_API_KEY") 
RESOURCE_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT") 

openai.api_type = "azure"
openai.api_key = API_KEY
openai.api_base = RESOURCE_ENDPOINT
openai.api_version = "2022-12-01"

url = openai.api_base + "/openai/deployments?api-version=2022-12-01"

r = requests.get(url, headers={"api-key": API_KEY})

print(r.text)

/anaconda/envs/azureml_py310_sdkv2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


{
  "data": [
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-davinci-002",
      "owner": "organization-owner",
      "id": "Davinci-002",
      "status": "succeeded",
      "created_at": 1677170437,
      "updated_at": 1677170437,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-ada-001",
      "owner": "organization-owner",
      "id": "ada-001",
      "status": "succeeded",
      "created_at": 1677170559,
      "updated_at": 1677170559,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"
      },
      "model": "text-search-curie-doc-001",
      "owner": "organization-owner",
      "id": "text-search-curie-doc-001",
      "status": "succeeded",
      "created_at": 1677549363,
      "updated_at": 1677549363,
      "object": "deployment"
    },
    {
      "scale_settings": {
        "scale_type": "standard"


In [3]:
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential

# Set the service endpoint and API key from the environment

service_name = "gios-cognitivesearch"#"YOUR-SEARCH-SERVICE-NAME"
admin_key = os.environ["AZURE_SEARCH_API_KEY"]#"YOUR-SEARCH-SERVICE-ADMIN-API-KEY"

index_name = "hotels-quickstart"

# Create an SDK client
endpoint = "https://{}.search.windows.net/".format(service_name)


search_client = SearchClient(endpoint=endpoint,
                      index_name=index_name,
                      credential=AzureKeyCredential(admin_key))

In [56]:
results

<iterator object azure.core.paging.ItemPaged at 0x7f91361b7100>

In [60]:
def build_prompt(QUERY,SEARCHRESULTS) : 

    return """Answer each question by extracting key facts from as many inputs paragraphs as possible.

    ###
    Input paragraphs:
    - Record numbers of solar panels were shipped in the United States during 2021. U.S. shipments of solar photovoltaic (PV) modules (solar panels) rose to a record electricity-generating capacity of 28.8 million peak kilowatts (kW) in 2021, from 21.8 million peak kW in 2020, based on data from our Annual Photovoltaic Module Shipments Report. Continued demand for U.S. solar capacity drove this increase in solar panel shipments in 2021. U.S. solar panel shipments include imports, exports, and domestically produced and shipped panels. In 2021, about 80% of U.S. solar panel module shipments were imports, primarily from Asia. U.S. solar panel shipments closely track domestic solar capacity additions; differences between the two usually result from the lag time between shipment and installation. We categorize solar capacity additions as either utility-scale (facilities with one megawatt of capacity or more) or small-scale (largely residential solar installations). The United States added 13.2 gigawatts (GW) of utility-scale solar capacity in 2021, an annual record and 25% more than the 10.6 GW added in 2020, according to our Annual Electric Generator Report. Additions of utility-scale solar capacity reached a record high, despite project delays, supply chain constraints, and volatile pricing. Small-scale solar capacity installations in the United States increased by 5.4 GW in 2021, up 23% from 2020 (4.4 GW). Most of the small- scale solar capacity added in 2021 was installed on homes. Residential installations totaled more than 3.9 GW in 2021, compared with 2.9 GW in 2020. The cost of solar panels has declined significantly since 2010. The average value (a proxy for price) of panel shipments has decreased from $1.96 per peak kW in 2010 to $0.34 per peak kW in 2021. Despite supply chain constraints and higher material costs in 2021, the average value of solar panels decreased 11% from 2020. In 2021, the top five destination states for U.S. solar panel shipments were: 1. California (5.09 million peak kW) 2. Texas (4.31 million peak kW) 3. Florida (1.80 million peak kW) 4. Georgia (1.15 million peak kW) 5. Illinois (1.12 million peak kW) These five states accounted for 46% of all U.S. shipments.
    - California fuel mix changes in response to September heat wave -10 0 10 20 30 40 50 60 Sep 4 Sep 5 Sep 6 Sep 7 Sep 8 Sep 9 Sep 10 CAISO electricity supply by source (Sep 4–Sep 10, 2022) gigawatts natural gas solar wind hydro imports all other nuclear batteries (discharging) batteries (charging) Data source: California Independent System Operator An extreme heat wave affected California the week of September 4, 2022, driving record-breaking demand for electricity to meet increased air-conditioning use. On September 6, a new record was set in the California Independent System Operator’s (CAISO) territory. CAISO, the grid operator for most of the state, issued appeals for consumer energy conservation throughout the week, as well as Energy Emergency Alerts each day, to help reduce electricity demand and prevent rolling power outages. CAISO predominately used natural gas, electricity imports, and hydroelectric sources during the highest demand hours to meet the record-breaking demand, which was a change from the typical mix. For brief periods during the week of September 4, CAISO used natural gas for as much as 60%—and never less than 30%—of the generation mix to meet electricity demand. California typically uses a mix of solar, wind, imports, hydroelectric, and natural gas sources for electricity generation. The exact mix depends on the time of day, the availability of sources, and the price that power plants set to sell electricity to the grid. This year, up to the record-setting demand week in September, CAISO’s generation mix included: 40% from solar, wind, nuclear, batteries, and other sources 32% from natural gas 20% from imports 7% from hydroelectric The mix relies slightly more on natural gas during the evening hours from 6:00 p.m. to 9:00 p.m., when electricity demand peaks and solar generation wanes. During the week of September 4, 2022, however, natural gas contributed nearly one-half of the resource mix in CAISO; nuclear, solar, wind, batteries, and other resources decreased to a 24% share. In California, natural gas units are often the last resource turned on to meet demand because they can be turned on after the sun sets in the evening when cooling demand remains high. When demand reaches record highs, seldom-used (less efficient, more expensive) natural gas units are needed to meet demand. 0% 25% 50% 75% 100% ...the week of Sep 4 from 5 p.m. to 9 p.m....this year CAISO's electricity generation mix so far... (Jan 1, 2022–Sept 12, 2022) percentage natural gas solar wind hydro batteries imports all other nuclear ...this year from 5 p.m. to 9 p.m. ...the week of Sep 4 Data source: California Independent System Operator Principal contributor: M. Tyson Brown
    - In 2020, 3.7% of U.S. single-family homes, including mobile homes, generated electricity from small-scale solar systems (solar panels installed on a home or building), according to our 2020 Residential Energy Consumption Survey (RECS). In 2018, 1.6% of U.S. commercial buildings had small-scale solar generation, according to our 2018 Commercial Energy Consumption Survey (CBECS). The presence of small-scale solar generation varied by a number of characteristics. Some of the largest differences were based on region. The West Census Region, where 8.9% of single-family homes (2020 data) had small-scale solar generation, had the highest percentage of homes with small-scale solar generation, mostly in California. Of commercial buildings in the West, 3.8% (48,000) had small-scale solar generation (2018 data). The Northeast Census Region had the second-largest percentage of homes with small-scale solar generation at 4.7% (2020 data). Similarly, for commercial buildings, 2.5% (20,000) had small-scale solar generation (2018 data). These two regions generally have had policies and incentives that have fostered growth of small-scale solar

    Question:
    Is small-scale solar growing? How much is it adding?

    Answer:
    Yes, it's growing. Record numbers of solar panels were shipped in the United States during 2021. U.S. shipments of solar photovoltaic (PV) modules (solar panels) rose to a record electricity-generating capacity of 28.8 million peak kilowatts (kW) in 2021, from 21.8 million peak kW in 2020, based on data from our Annual Photovoltaic Module Shipments Report. In 2020, 3.7% of U.S. single-family homes, including mobile homes, generated electricity from small-scale solar systems (solar panels installed on a home or building), according to our 2020 Residential Energy Consumption Survey (RECS).

    ###
    Input paragraphs:
    {}

    Question:
    {}

    Answer:
    """.format(SEARCHRESULTS,QUERY)


In [69]:
print(SearchResults)

- The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services. 
 - Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace. 
 - The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities. 
 - The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique 

In [83]:
query="What can you tell me about the hotel?"

results =  search_client.search(search_text=query, include_total_count=True, search_mode="any")

print ('Total Documents Matching Query:', results.get_count())

SearchResults = ""

for result in results:
    print("{}: {}: {}".format(result["HotelId"], result["HotelName"] , result["Description"] ))
 
    SearchResults += "- "+result["Description"]+" \n " 

# prompt = '{\
#     "QUERY":'+query+', \
#     "SearchResults": '+SearchResults+'\
#     }Tl;dr'

prompt = build_prompt(query, SearchResults)

# prompt = SearchResults + "Tl;dr"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop=".", temperature=0,
                                     echo=False,max_tokens= 100,
)
print("-----------------Result-----------------")                      
# print(f"{prompt}{completion['choices'][0]['text']}.")

print(f"{completion['choices'][0]['text']}.")


Total Documents Matching Query: 4
3: Triple Landscape Hotel: The Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services.
4: Sublime Cliff Hotel: Sublime Cliff Hotel is located in the heart of the historic center of Sublime in an extremely vibrant and lively area within short walking distance to the sites and landmarks of the city and is surrounded by the extraordinary beauty of churches, buildings, shops and monuments. Sublime Cliff is part of a lovingly restored 1800 palace.
1: Secret Point Motel: The hotel is ideally located on the main commercial artery of the city in the heart of New York. A few minutes away is Time's Square and the historic centre of the city, as well as other places of interest that make New York one of America's most attractive and cosmopolitan cities.
2: Twin Dome Motel: The hotel is situated in a  nineteenth century plaza, which has been expanded and renovated to

In [66]:
completion

<OpenAIObject text_completion id=cmpl-6p1XOtm6AztrkeicwLP7hmMUFMW3j at 0x7f9136322430> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThe Hotel stands out for its gastronomic excellence under the management of William Dough, who advises on and oversees all of the Hotel's restaurant services"
    }
  ],
  "created": 1677618658,
  "id": "cmpl-6p1XOtm6AztrkeicwLP7hmMUFMW3j",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 31,
    "prompt_tokens": 1785,
    "total_tokens": 1816
  }
}

In [6]:
def concatenate_values(dictionary, string):
    descriptions = [result[string] for result in dictionary]
    return " ".join(descriptions)

prompt = concatenate_values(results,"HotelName")
prompt

''

In [ ]:
summary = ""

#Get completions

Query = ""
SearchResults = ""



In [22]:
prompt = "The food was delicious and the waiter"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")


The food was delicious and the waiter was very attentive.


In [58]:
prompt = "Say this is a test"
completion = openai.Completion.create(deployment_id="Davinci-002",
                                     prompt=prompt, stop=".", temperature=0)
                                
print(f"{prompt}{completion['choices'][0]['text']}.")

Say this is a test

This is a test.
